# Трансформеры
В этом домашнем задании мы рассмотим использование трансформеров в библиотеке PyTorch. Рассмотрим задачу языкового моделирования. Попробуем генерировать текст нейронной сетью. 

Ссылка на данные - https://drive.google.com/drive/folders/1x1A4ElliUGBPnHladGMwPxPuGxI8Vnpu?usp=sharing

In [ ]:
# хороший тон, импортировать все необходимые библиотеки в одной ячейке ;)

import torch
from torch import nn

import numpy as np
import time
from tqdm.notebook import tqdm

Что такое языковое моделирование? Это предсказание вероятности следующего токена (слова или буквы) на основе предыдущих токенов. Математически это можно описать так:

$$P(x_i|x_1, x_2 , ... , x_{i-1})$$ 

Последовательность $$ x_1, x_2, ... x_{i-1} $$ называют контекстом.

## Задание 0 (0 баллов, но сделать нужно)
Проставьте знаки неравенств, исходя из вашего опыта:
$$ P(раму | мама, мыла) > P(папу | мама, мыла) $$
$$ P(столу | дорога, ложка, к) < P(обеду | дорога, ложка, к) $$
$$ P(Евпатий | меня, зовут) < P(Ваня | меня, зовут) $$
$$ P(журналы | я, часто ,читаю) > P(комиксы | я, часто ,читаю) $$
Попробуйте объяснить выбор для каждого из примеров.

Ответ :
- встретить слово рама среди слов мама и мыла вероятнее, так как это поговорка или что-то такое
- такой же принцип. Дорога ложко к обеду – фразеологизм
- Ваня – более распространенное имя, чем Евпатий
- не читаю комиксы, поэтому скажу, что тут знак "больше"

Если для некоторых из примеров проставить знаки достаточно просто, то на некоторые сложно сказать, какой овтет верный. Мы принимаем решение для данного задания исходя их опыта использования русского языка. Мы много читали на русском и слушали огромное количество русской речи. Обучение языковых моделей происходит по схожему принципу. 

Мы хотим показать модели столько текстов, сколько можем и надеемся, что она наберется достаточно опыта, чтобы расставлять такие знаки неравества максимально схоже с человеком.

## Задание 1 (0.5 балла)
Мы будем обучать языковую модель для предсказания следущей буквы. Такие языковые модели применяются в распозновании речи, так как предоставляют дополнительную информацию акустической модели при выборе следующего символа. Для начала, откройте файл с данными, посмотрите, какие символы входят в тексты, сколько их. Уберите из текста все символы переноса на новую строку и табуляцию.

In [3]:
path = '../input/hw3transformer/small_corp_for_test.txt'
file = open(path, 'r')
data = file.readlines()
file.close()
len(data)

700000

In [4]:
data = [text.strip('\n') for text in data]

## Задание 2 (0.5 балла)
Для обучения модели требуется сначала подготовить текст в подходящий для нейросети вид. Важно также отметить, что нужно добавить два токена start и end, которые отвечают за начало и конец текста. Используйте [ и ] для этой задачи. Также нам нужен токен pad, чтобы заполнять им текст до требуемой длинны для формирования батча.

Реализуйте метод preprocess класса Preprocessor. Он должен принимать на вход текст и длинну текста, которую мы ожидаем получить на выходе. Текст должен быть переведен в нижний регистр, в конец текста добавляется требуемое число pad токенов, далее текст векторизуется (каждому символу ставится свое число). Вернуть требуется два вектора. Полученный результат без последнего токена (на нем будем обучаться) и полученный результат без первого токена (целевые метки при обучении).

In [5]:
class Preprocessor:
    def __init__(self):
        self.alphabet = '_добсркгаупитнезчмфяжлйвцыэь-шхющёъ][ '
        self.token2ind = {}
        self.ind2token = {}
        for i in range(len(self.alphabet)):
            self.token2ind[self.alphabet[i]] = i
            self.ind2token[i] = self.alphabet[i]
        
    
    def preprocess(self, text, window_size):
        text = str.lower(text)
        pads = window_size - len(text)
        text = f'[{text}]'
        text += '_' * pads
        v1 = [self.token2ind[char] for char in text[1:]]
        v2 = [self.token2ind[char] for char in text[:-1]]
        return [v1, v2]

## Задание 3 (0.5 балла)
Так как мы решили, что текст будет начинаться токеном [ и заканчиваться токеном ], данные нужно поправить. Реализуйте эту идею, добавьте данные токены в ваши тексты.

это задание сделано в прошлом пункте

## Задание 4 (0.5 балла)
Так как мы не располагаем большими мощностями, то давайте ограничим максимальную длинну текста. Вы можете менять этот порог и тем самым уменьшать кол-во текстов в вашей выборке и увеличивая тем самым скорость обучения. Начнем же мы с 128. 
Выберите порог и оставьте только те тексты, длина которых не превосходит данный порог.

Далее разбейте тексты на train и test, перемешайте тексты при разбиении, размер тестовой выборки должен быть 15% от общего числа текстов. 

In [6]:
max(map(len, data))

540

In [7]:
THRESHOLD = 127

data = [text for text in data if len(text) <= THRESHOLD]

In [8]:
from sklearn.model_selection import train_test_split

data_train, data_test = train_test_split(data, random_state=777, test_size=0.15)

## Задание 5 (2 балла)
Напишем датасет. На вход датасету передается набор текстов, объект класса Preprocessor и размер окна, который вы выбрали в прошлом задании.
Реализуйте методы __len__ и __getitem__.

In [9]:
class TextDataset(torch.utils.data.Dataset):
    
    def __init__(self, x, preproc, win_size = 127):
        self.x = x
        self.preproc = preproc
        self.win_size = win_size
    
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        x, y = self.preproc.preprocess(self.x[idx], self.win_size)
        x = torch.LongTensor(x)
        y = torch.LongTensor(y)
        return x, y

In [10]:
preproc = Preprocessor()
train_dataset = TextDataset(data_train, preproc)
test_dataset = TextDataset(data_test, preproc)

## Задание 6 (2 балла)
Напишем модель. Класс для реализации positional encoding реализован за вас, он нужен, чтобы модель могла после получения эмбедингов понимать, на каком месте какой токен находится.

Заполните пропуски в классе модели. Гипперпараметры модели вам предлагается подобрать самостоятельно. Рекомендуется использовать не более 6 слоев в трансформере. В декореде испоьлзуйте две линейных слоя с функцией активации ReLU между ними.

## Задание 6_1 (0 баллов, но надо ответить!)
При обучении языковой модели на основе трансформеров мы используем маскирование символов (как мы это делаем - уже реализовано). Напишите, почему мы это делаем? Почему это так важно?

- Это важно, так как модель предсказывает выход для каждого символа. Не скрывая следующий вход данных модель будет подсматривать вперед и в итоге переобучится

In [11]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [12]:
import math
from typing import Tuple

import torch
from torch import nn, Tensor
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.utils.data import dataset

In [13]:
class LanguageModel(nn.Module):
    def __init__(self, vocab_size, dropout=0.1):
        super(LanguageModel, self).__init__()
        self.emb = nn.Embedding(vocab_size, 32, padding_idx=37)
        self.pe = PositionalEncoding(32, dropout)
        self.transformer_encoder_layer = TransformerEncoderLayer(32, 8)
        self.transformer_encoder = TransformerEncoder(self.transformer_encoder_layer, 6)
        self.decoder = nn.Sequential(nn.Linear(32, 64),
                                     nn.ReLU(),
                                     nn.Linear(64, vocab_size))
    
    def forward(self, x, src_mask):
        x = self.pe.forward(self.emb(x)) # emb, then pe
        x = x.transpose(1, 0)
        x = self.transformer_encoder(x, src_mask) # transformer encoder with mask
        x = self.decoder(x) # decoder
        return x.transpose(1, 0)
    
    def generate_square_subsequent_mask(self, sz):
        # А вот и то самое маскирование
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

## Задание 7 (2,5 балла)
Финишная прямая. Давайте реализуем класс для обучения модели и ее валидации. Следуйте указаниям в коде и заполните недостающие фрагменты в коде.

In [54]:
class Trainer:
    
    def __init__(self, model, train_dataset, test_dataset):
        
        self.model = model
        
        self.train_batch_size = 64
        self.test_batch_size = 64
        
        self.train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=self.train_batch_size)
        self.test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=self.test_batch_size)
        self.train_dataloader_size = len(train_dataset)
        self.test_dataloader_size = len(test_dataset)
        
        self.device = 'cuda:0'
        self.criterion = nn.CrossEntropyLoss(ignore_index=0) # используйте CrossEntrophyLoss, передайте в качетсве параметра 
                                                             # ignore index индекс символа _, чтобы модель не штрафовалась за то
                                                             # что идет после закрывающего токена
        
        self.optimizer = torch.optim.Adam(model.parameters(), lr=0.001, eps=1e-08)
        
        self.steps_to_print = 1000
        
    def train_one_epoch(self, epoch_number):
        step = 0
        counted_loss = 0
        current_time = time.time()
        it = 0
        
        for batch in tqdm(self.train_dataloader):
            x, y = batch
            x = x.to(self.device)
            y = y.to(self.device)
            src_mask = self.model.generate_square_subsequent_mask(128).to(self.device)
            y_pred = self.model(x, src_mask)
            y_pred = torch.reshape(y_pred, (y_pred.shape[0]*128, 38))
            y = y.flatten()
            loss = self.criterion(y_pred, y)
            loss.backward()
            self.optimizer.step()
            self.optimizer.zero_grad()
            counted_loss = float(loss)
            step += 1
            it += 1
            
            
            if step % self.steps_to_print == 0:
                result = 'Train epoch '+str(epoch_number)+' | '
                result += 'Step '+str(step)+'/'+str(self.train_dataloader_size)+' | '
                result += 'Counted loss '+str(counted_loss)+' | '
                result += 'ppl '+str(math.exp(counted_loss/it))+' | '
                result += 'time '+str(time.time() - current_time) + ' | '
                print(result)
                current_time = time.time()
                counted_loss = 0
                it = 0
    
    def validate_one_epoch(self, epoch_number):
        step = 0
        counted_loss = 0
        current_time = time.time()
        it = 0
        for batch in tqdm(self.test_dataloader):
            x, y = batch
            x = x.to(self.device)
            y = y.to(self.device)
            src_mask = self.model.generate_square_subsequent_mask(128).to(self.device)
            y_pred = self.model(x, src_mask)
            y_pred = torch.reshape(y_pred, (y_pred.shape[0]*128, 38))
            y = y.flatten()
            loss = self.criterion(y_pred, y)
            counted_loss = float(loss)
            step += 1
            it += 1
            
            
            if step%(self.steps_to_print // 2) == 0:
                result = 'Validate epoch '+str(epoch_number)+' | '
                result += 'Step '+str(step)+'/'+str(self.test_dataloader_size)+' | '
                result += 'Counted loss '+str(counted_loss)+' | '
                result += 'ppl '+str(math.exp(counted_loss/it))+' | '
                result += 'time '+str(time.time() - current_time) + ' | '
                print(result)
                current_time = time.time()
                counted_loss = 0
                it = 0
        
    def train(self, number_of_epochs):
        model.to(self.device)
        for epoch in range(1, number_of_epochs+1):
            model.train()
            self.train_one_epoch(epoch)
            with torch.no_grad():
                model.eval()
                self.validate_one_epoch(epoch)
            print()

Что такое ppl? Перплексия. Ее можно интерпретировать как меру "удивленности" модели нужному символу. Чем меньше данная величина, тем лучше, ведь это значит, что модель если и сделала неправильный выбор, то не сильно удивлена своей ошибке.

Проведите несколько экспериментов, посмотрите, при каких гипперпараметрах значение перплексии минимально.

## Задание 8 (0.5 балла)
Запустите обучение на нескольких эпохах. Ориентируйтесь на ваши вычислительные мощности и время работы. Вы всегда можете посчитать, сколько секунд уходит на один батч.

In [55]:
model = LanguageModel(len('_добсркгаупитнезчмфяжлйвцыэь-шхющёъ][ '), dropout=0.1)

In [56]:
trainer = Trainer(model, train_dataset, test_dataset)

In [105]:
trainer.train(2)

  0%|          | 0/9082 [00:00<?, ?it/s]

Train epoch 1 | Step 1000/581190 | Counted loss 0.02129022218286991 | ppl 1.0000212904488213 | time 53.958184003829956 | 
Train epoch 1 | Step 2000/581190 | Counted loss 0.011111130937933922 | ppl 1.0000111111926668 | time 53.66317272186279 | 
Train epoch 1 | Step 3000/581190 | Counted loss 0.014386584982275963 | ppl 1.0000143866884696 | time 53.65493154525757 | 
Train epoch 1 | Step 4000/581190 | Counted loss 0.014124536886811256 | ppl 1.0000141246366385 | time 53.707499265670776 | 
Train epoch 1 | Step 5000/581190 | Counted loss 0.02209186740219593 | ppl 1.0000220921114293 | time 53.66835927963257 | 
Train epoch 1 | Step 6000/581190 | Counted loss 0.018644412979483604 | ppl 1.0000186445867876 | time 53.657555103302 | 
Train epoch 1 | Step 7000/581190 | Counted loss 0.020717110484838486 | ppl 1.0000207173250857 | time 53.724501848220825 | 
Train epoch 1 | Step 8000/581190 | Counted loss 0.010232968255877495 | ppl 1.0000102330206129 | time 53.56143355369568 | 
Train epoch 1 | Step 9000

  0%|          | 0/9082 [00:00<?, ?it/s]

Train epoch 2 | Step 1000/581190 | Counted loss 0.0135105075314641 | ppl 1.0000135105987988 | time 53.84310293197632 | 
Train epoch 2 | Step 2000/581190 | Counted loss 0.009610163979232311 | ppl 1.000009610210157 | time 53.70690989494324 | 
Train epoch 2 | Step 3000/581190 | Counted loss 0.003650265745818615 | ppl 1.0000036502724081 | time 53.72431015968323 | 
Train epoch 2 | Step 4000/581190 | Counted loss 0.011499692685902119 | ppl 1.0000114997588077 | time 53.71136140823364 | 
Train epoch 2 | Step 5000/581190 | Counted loss 0.0072856987826526165 | ppl 1.0000072857253235 | time 53.57168483734131 | 
Train epoch 2 | Step 6000/581190 | Counted loss 0.01093230303376913 | ppl 1.0000109323627917 | time 53.59477686882019 | 
Train epoch 2 | Step 7000/581190 | Counted loss 0.005390532314777374 | ppl 1.0000053905468438 | time 53.81950759887695 | 
Train epoch 2 | Step 8000/581190 | Counted loss 0.0067556253634393215 | ppl 1.0000067556481826 | time 54.04359459877014 | 
Train epoch 2 | Step 9000/

## Задание 9 (1 балл)
Итак, давайте попробуем погенерировать текст нашей сеткой. Закончите функцию по генерации текста. Попробуйте сгенерировать какой-нибудь текст. Помните, что если вы хотите генерировать текст с нуля, то вы должны передать в качестве текста только токен start.
Прекратите генерировать текст, если модель выдала токен end или длинна текста больше 150.

In [76]:
mask = model.generate_square_subsequent_mask(128).to('cuda:0')

In [95]:
def generate_text(text):
    x = []
    device = 'cuda:0'
    
    for letter in text:
        x.append(preproc.token2ind[letter])
    x = torch.from_numpy(np.array(x))
    return x
    x = x.to(device)
    
    pred = model(x, )
    ind = ...
    
    text += ... 
    
    if ...:
        return text
    else:
        return generate_text(text)